In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [2]:
data = pd.read_table('train.txt',sep=' ',index_col=False) 

In [3]:
data2 = pd.read_csv('1234.csv',index_col=False) 

In [4]:
data.pop('item_category_list')
data.insert(1, 'item_category_list', data2[['item_category_list']])
data.pop('item_property_list')
data.insert(2, 'item_property_list', data2[['item_property_list']])
pass

In [5]:
data[['context_timestamp']] = data[['context_timestamp']] - 1537200000

In [6]:
X = data[['item_category_list','item_property_list',
          'shop_review_positive_rate','shop_score_service',
         'shop_score_delivery','shop_score_description','shop_star_level'
          ,'shop_review_num_level','item_price_level','item_sales_level',
         'item_collected_level','item_pv_level',
          'user_gender_id','user_age_level',
          'user_occupation_id','user_star_level',
          'context_page_id','context_timestamp']]
Y = data[['is_trade']]
X = np.array(X)
Y = np.array(Y).ravel()

In [13]:
# 制作训练集与测试集并且进行预处理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=666)

In [12]:
X_shop_train = X_train[:, 2:8].copy()
X_item_train = X_train[:, [0,1,8,9,10,11]].copy()
X_user_train = X_train[:, 12:15].copy()
X_content_train = X_train[:, [0,1,15,16]].copy()
X_shop_test = X_test[:, 2:8].copy()
X_item_test = X_test[:, [0,1,8,9,10,11]].copy()
X_user_test = X_test[:, 12:15].copy()
X_content_test = X_test[:, [0,1,15,16]].copy()

In [29]:
def pi(degree=1,C=1,penalty='l2'):
    poly_arg = [
        ("poly", PolynomialFeatures(degree=degree)),
        ("std_scaler", StandardScaler()),
        ("log_clf", LogisticRegression(C=C,penalty='l2'))
    ]
    poly_reg = Pipeline(poly_arg)
    return poly_reg

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [91]:
log_clf1 = pi(C = 1,penalty='l2',degree=2)

In [92]:
log_clf1.fit(X_shop_train, Y_train)

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('log_clf', LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [93]:
log_loss(Y_train, log_clf1.predict_proba(X_shop_train)[:,1])

0.093315723033477224

In [94]:
log_loss(Y_test, log_clf1.predict_proba(X_shop_test)[:,1])

0.091524252839089104

In [137]:
log_clf2 = pi(degree=2).fit(X_item_train+1, Y_train)

In [138]:
log_loss(Y_train, log_clf2.predict_proba(X_item_train+1)[:,1])

0.091213476268405244

In [139]:
log_loss(Y_test, log_clf2.predict_proba(X_item_test+1)[:,1])

0.089508812735154292

In [161]:
log_clf3 = pi(C = 1,penalty='l1',degree=1).fit(X_user_train, Y_train)

In [162]:
log_loss(Y_train, log_clf3.predict_proba(X_user_train)[:,1])

0.09392730013442166

In [163]:
log_loss(Y_test, log_clf3.predict_proba(X_user_test)[:,1])

0.091887707773910338

In [182]:
log_clf4 = pi(C = 1,penalty='l1',degree=1).fit(X_content_train, Y_train)

In [183]:
log_loss(Y_train, log_clf4.predict_proba(X_content_train)[:,1])

0.093940554391314074

In [184]:
log_loss(Y_test, log_clf4.predict_proba(X_content_test)[:,1])

0.091974402182018591

In [194]:
last = LogisticRegression(penalty='l1', C=0.01)

c_data = log_clf1.predict_proba(X_shop_train)[:,1].reshape(-1, 1)
c_data = np.hstack((c_data, log_clf2.predict_proba(X_item_train)[:,1].reshape(-1, 1)))
c_data = np.hstack((c_data, log_clf3.predict_proba(X_user_train)[:,1].reshape(-1, 1)))
c_data = np.hstack((c_data, log_clf4.predict_proba(X_content_train)[:,1].reshape(-1, 1)))
last.fit(c_data,Y_train)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [195]:
c_data = log_clf1.predict_proba(X_shop_test)[:,1].reshape(-1, 1)
c_data = np.hstack((c_data, log_clf2.predict_proba(X_item_test)[:,1].reshape(-1, 1)))
c_data = np.hstack((c_data, log_clf3.predict_proba(X_user_test)[:,1].reshape(-1, 1)))
c_data = np.hstack((c_data, log_clf4.predict_proba(X_content_test)[:,1].reshape(-1, 1)))
log_loss(Y_test, last.predict_log_proba(c_data)[:,1])

0.63827462296697879

In [214]:
temp = ((c_data/4)*[0.1, 0., 0.1, 0.1]).sum(axis=1)
log_loss(Y_test,temp)

0.090417947817129751

In [209]:
((c_data/4)*[0.3, 0.1, 0.3, 0.3]).sum(axis=1)

array([ 0.00377242,  0.0046151 ,  0.0038897 , ...,  0.00555617,
        0.00443789,  0.00454203])